In [ ]:
import com.google.gson.GsonBuilder
import com.google.gson.TypeAdapter
import com.google.gson.stream.JsonReader
import com.google.gson.stream.JsonWriter
import hu.akos.hollo.szabo.math.Permutation
import hu.raven.puppet.logic.statistics.edgeHistogramMatrix
import hu.raven.puppet.model.state.BacteriophageAlgorithmState
import hu.raven.puppet.model.utility.math.GraphEdge


In [ ]:
object PermutationTypeAdapter : TypeAdapter<Permutation>() {
        override fun write(out: JsonWriter, value: Permutation) {
            out.beginArray()
            value.forEach { out.value(it) }
            out.endArray()
        }

        override fun read(input: JsonReader): Permutation {
            input.beginArray()
            val values = buildList {
                while (input.hasNext()) {
                    add(input.nextInt())
                }
            }
            input.endArray()
            return Permutation(values)
        }
    }

In [ ]:
fun <T> mapStates(
    filePath: String,
    mapper: (BacteriophageAlgorithmState<*>) -> T
): List<T> {
    val file = File(filePath)
    val gson = GsonBuilder()
        .registerTypeAdapter(Permutation::class.java, PermutationTypeAdapter)
        .create()

    return file.useLines { lines ->
        lines
            .chunked(10)
            .map { it.first() }
            .mapIndexed { index, line ->
                println(index)
                gson.fromJson(line, BacteriophageAlgorithmState::class.java)
            }
            .map { mapper(it) }
            .toList()
    }
}

In [ ]:
val edgeAccuranceMatrices = mapStates(
    "../output/2024-03-01/2024-03-01T18/algorithmState.json"
){    edgeHistogramMatrix(it) }

In [ ]:
%use kandy
%use dataframe

In [ ]:
val state = edgeAccuranceMatrixes[731]

val mappedToEdges = state
    .flatMapIndexed { sourceNodeIndex, edgesFromNode ->
        edgesFromNode.mapIndexed { targetNodeIndex, value ->
            GraphEdge(
                sourceNodeIndex = sourceNodeIndex,
                targetNodeIndex = targetNodeIndex,
                value = value
            )
        }
    }

val edgeRecurrence = dataFrameOf(
    "sourceNodeIndex" to mappedToEdges.map { it.sourceNodeIndex },
    "targetNodeIndex" to mappedToEdges.map { it.targetNodeIndex },
    "numberOfEdges" to mappedToEdges.map { it.value },
)

In [ ]:
edgeRecurrence.plot {
    x(sourceNodeIndex)
    y(targetNodeIndex)
    heatmap(sourceNodeIndex, targetNodeIndex) {
        fillColor(numberOfEdges.map { if (it == 0) 1 else 0 }) {
            scale = continuous(Color.WHITE..Color.RED)
            legend.name = "number of edges"
        }
    }
}

In [ ]:
edgeRecurrence.plot {
    x(sourceNodeIndex)
    y(targetNodeIndex)
    histogram(numberOfEdges, binsOption = BinsOption.byWidth(1.0)) 
}

In [ ]:
val bestValuesData = mapStates(
    "../output/2024-03-01/2024-03-01T18/algorithmState.json"
) { 
    it.population.activesAsSequence()
        .minOf { it.cost?.get(0) ?: -1f } 
}

In [ ]:
val bestCostTable = dataFrameOf( 
    "indexes" to bestValuesData.indices.toList(),
    "bestCost" to bestValuesData,
)

In [ ]:
bestCostTable.plot { 
    line { 
        x(indexes)
        y(bestCost)
        color = Color.BLUE
    }
}

In [ ]:
val costsData  = mapStates(
    "../output/2024-03-04/2024-03-04T22/algorithmState.json"
) {
    it.population.activesAsSequence()
        .map { it.cost?.get(0) ?: -1f }
        .toList()
        .toFloatArray()
}

In [ ]:
val costsTable = dataFrameOf(
    "index" to costsData.indices.toList(),
    "max" to costsData.map { it.max() },
    "avg" to costsData.map { it.average() },
    "min" to costsData.map { it.min() },
    "spread" to costsData
        .map { costs ->
            costs
                .map { abs(it - costs.average()) }
                .average()
        },
    "mean" to costsData
        .map { it.sorted()[it.size / 2] }
)

In [ ]:
costsTable.plot {
    x(index)
    line {
        y(max)
        color = Color.BLUE
    }
    line {
        y(min)
        color = Color.RED
    }
    line {
        y(avg)
        color = Color.PURPLE
    }
    line {
        y(spread)
        color = Color.GREEN
    }
    line {
        y(mean)
        color = Color.YELLOW
    }
}